# Comparison with SIB on SI model
SIB: https://www.nature.com/articles/srep27538, https://github.com/sibyl-team/sib

In [1]:
using MatrixProductBP, MatrixProductBP.Models
using Graphs, Plots, Printf, IndexedGraphs, Statistics
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

In [2]:
using PyCall, SparseArrays

ENV["PYTHON"] = "/home/crotti/miniconda3/bin/python"

@pyimport sib

# convert ϕ's into observations for sib
function sib_SI_tests(ϕ::Vector{Vector{Vector{F}}}) where F
    tests = []
    for i in eachindex(ϕ)
        for t in 2:lastindex(ϕ[i])
            ϕᵢᵗ = ϕ[i][t]
            # uniform <=> no observation
            if all(isequal(ϕᵢᵗ[1]), ϕᵢᵗ)
                nothing
            else
                push!(tests, (i-1, sib.Test(ϕᵢᵗ..., 0), t+1))
            end
        end
    end
    tests
end


function sib_SI(T::Integer, g::IndexedGraph, ϕ, λ;
        maxiter = 400, tol = 1e-14)
    γ = ϕ[1][1][INFECTIOUS]
    @assert all(isequal(γ), ϕᵢ[1][INFECTIOUS] for ϕᵢ in ϕ)

    contacts = [(src(e)-1, dst(e)-1, t, λ) for t in 1:T for e in edges(g)]
    append!(contacts, [(dst(e)-1, src(e)-1, t, λ) for t in 1:T for e in edges(g)])
    sort!(contacts, by=x->x[3])
    tests = sib_SI_tests(ϕ)
    params = sib.Params(prob_r=sib.Exponential(1e-100), pseed=γ/(2-γ), psus=(1-γ)/(2-γ), pautoinf=0.0)
    f = sib.FactorGraph(contacts=contacts, tests=tests, params=params)
    sib.iterate(f, maxit=maxiter, tol=tol)
    sib.iterate(f, maxit=maxiter, damping=0.5, tol=tol)
    sib.iterate(f, maxit=maxiter, damping=0.9, tol=tol)

    marginals_sib = map(f.nodes) do i
        map(1:T) do t 
            m = sib.marginal_t(i, t)
            @assert m[3] ≈ 0
            [m[1], m[2]]
        end
    end

    F_bethe = - f.loglikelihood() - N*log(2-γ)

    return marginals_sib, F_bethe
end

sib_SI (generic function with 1 method)

In [3]:
T = 10
N = 30
k = 3
gg = random_regular_graph(N, k)
g = IndexedGraph(gg)
λ = 0.15
ρ = 0.0   # SI
γ = 0.2
sis = SIS(g, λ, ρ, T; γ);

In [4]:
bp = mpbp(sis)
nobs = N
draw_node_observations!(bp, nobs, last_time=true)
cb = CB_BP(bp);

In [5]:
svd_trunc = TruncBond(4)
iters, cb = iterate!(bp, maxiter=100; svd_trunc, cb, tol=1e-10);

Running MPBP: iter 2    Time: 0:00:21
  Δ:                0.6193901777367918




Running MPBP: iter 3    Time: 0:00:22
  Δ:                0.13416424532877458




Running MPBP: iter 4    Time: 0:00:22
  Δ:                0.12910112512771366




Running MPBP: iter 5    Time: 0:00:23
  Δ:                0.0341918319998038




Running MPBP: iter 6    Time: 0:00:23
  Δ:                0.006587603729064639




Running MPBP: iter 7    Time: 0:00:23
  Δ:                0.0010184603201321973




Running MPBP: iter 8    Time: 0:00:23
  Δ:                0.0006069333114107067




Running MPBP: iter 9    Time: 0:00:24
  Δ:                7.532375353247467e-5




Running MPBP: iter 10    Time: 0:00:24
  Δ:                6.416556752486535e-5




Running MPBP: iter 11    Time: 0:00:24
  Δ:                1.1209822249114154e-5




Running MPBP: iter 12    Time: 0:00:25
  Δ:                5.102351408492467e-6




Running MPBP: iter 13    Time: 0:00:25
  Δ:                7.65263730162502e-7




Running MPBP: iter 14    Time: 0:00:25
  Δ:                5.567581933263455e-7




Running MPBP: iter 15    Time: 0:00:25
  Δ:                7.00706326295375e-8




Running MPBP: iter 16    Time: 0:00:26
  Δ:                4.965577105053853e-8




Running MPBP: iter 17    Time: 0:00:26
  Δ:                6.160755017248221e-9




Running MPBP: iter 18    Time: 0:00:26
  Δ:                5.224678467641297e-9




Running MPBP: iter 19    Time: 0:00:27
  Δ:                5.838414196546182e-10




Running MPBP: iter 20    Time: 0:00:27
  Δ:                4.564608691026706e-10




Running MPBP: iter 21    Time: 0:00:27
  Δ:                5.725198093387007e-11


In [6]:
pl_err = plot(cb.Δs, ylabel="convergence error", xlabel="iters", m=:o, yaxis=:log10, 
    size=(300,200), label="")

In [7]:
fe_bp = bethe_free_energy(bp)
b_bp = beliefs(bp)
p_bp = [[bbb[2] for bbb in bb] for bb in b_bp];

Run sib

In [8]:
b_sib, fe_sib = sib_SI(T, g, bp.ϕ, λ)
p_sib = [[bbb[2] for bbb in bb] for bb in b_sib];

sib.iterate(damp=0.0): 54/400 9.548e-15/1e-14      
sib.iterate(damp=0.5): 0/400 8.216e-15/1e-14      


SI on random regular graph N=30, k=3. P(S→I)=0.15.
30 random observations.
Max matrix size: 4x4


In [9]:
pl_sib = plot(xlabel="time", ylabel="\$P(x_i=I)\$", xticks=0:2:T-1, title="sib")
for i in 1:N
    plot!(pl_sib, 0:T-1, p_sib[i], label="i=$i", m=:o, ms=3, lw=1)
end
pl_bp = plot(xlabel="time", ylabel="\$P(x_i=I)\$", xticks=0:2:T-1, title="MPdBP")
for i in 1:N
    plot!(pl_bp, 0:T-1, p_bp[i][1:T], label="i=$i", m=:o, ms=3, lw=1)
end
println("SI on random regular graph N=$N, k=$k. P(S→I)=$λ.")
println("$nobs random observations.")
println("Max matrix size: ", svd_trunc.mprime, "x", svd_trunc.mprime)
pl_sc = scatter(reduce(vcat, [p[1:end-1] for p in p_bp]), reduce(vcat, p_sib), xlabel="MPdBP", 
    ylabel="sib", label="\$P(x_i=I)\$",  ms=3, c=:black, legend=:outertopright)
plot!(pl_sc, identity, label="", size=(300,300))
plot(pl_sib, pl_bp, pl_sc, titlefontsize=10, size=(950, 300), legend=:outertopright, 
    margin=5Plots.mm, layout=(1,3))

Compare estimates for the free energy

In [10]:
fe_bp, fe_sib

(13.08834313573998, 13.088937216860288)

In [11]:
(fe_bp - fe_sib) / fe_sib

-4.538803345645028e-5